## Моделирование газовой динамики (распад произвольного разрыва)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation, ArtistAnimation
from tqdm import tqdm_notebook
from numba import jit

### Метод крупных частиц (МКЧ)

In [2]:
@jit
def P(ro, u, e):
    return ro*(e-u**2/2)*(gamma-1)


@jit
def nextLayer(ro, u, e):
    """
    ro, u, e - numpy arrays at previuos layer
    returns array with new numpy arrays
    """
    rv = ro.copy()
    uv = u.copy()
    ev = e.copy()
    
    new_ro = ro.copy()
    new_u = u.copy()
    new_e = e.copy()
    
    for i in range(1, N-1):
        pp = (P(ro[i], u[i], e[i]) + P(ro[i+1], u[i+1], e[i+1]))/2
        pn = (P(ro[i], u[i], e[i]) + P(ro[i-1], u[i-1], e[i-1]))/2
        
        up = (u[i] + u[i+1])/2
        un = (u[i] + u[i-1])/2
        
        uv[i] -= r*(pp-pn)/ro[i]
        ev[i] -= r*(pp*up-pn*un)/ro[i]
        
    for i in range(1, N-1):
        uvp = (uv[i] + uv[i+1])/2
        uvn = (uv[i] + uv[i-1])/2
        
        Dp = 0 if uvp >= 0 else 1
        Dn = 1 if uvn >= 0 else 0
        
        Yp = ro[i]*uvp if uvp >= 0 else -ro[i+1]*uvp
        Yn = ro[i-1]*uvn if uvn >= 0 else -ro[i]*uvn
        
        new_ro[i] = ro[i] +2*r*((Dn-0.5)*Yn + (Dp-0.5)*Yp)
        new_u[i] = uv[i]*ro[i]/new_ro[i] + r/new_ro[i]*(uv[i]*((Dn-1)*Yn+(Dp-1)*Yp)+uv[i-1]*Dn*Yn+uv[i+1]*Dp*Yp)
        new_e[i] = ev[i]*ro[i]/new_ro[i] + r/new_ro[i]*(ev[i]*((Dn-1)*Yn+(Dp-1)*Yp)+ev[i-1]*Dn*Yn+ev[i+1]*Dp*Yp)
        
    return [new_ro, new_u, new_e]

In [21]:
dt = 0.0004
dx = 0.004

N = 1000
T = 500

r = dt/dx
gamma = 1.5

ro = np.ones(N)
u = np.ones(N)
e = np.ones(N)

ro[:N//2] = [1.5]*(N//2)
ro[N//2:] = [1]*(N//2)

u[:N//2] = [0]*(N//2)
u[N//2:] = [0]*(N//2)

e[:N//2] = [2]*(N//2)
e[N//2:] = [1]*(N//2)


fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (13, 10))
xdata = dx*np.array(range(N))
ln1, = ax[0].plot([], [], color = 'r', linestyle = '-', label = 'Плотность')
ln2, = ax[1].plot([], [], color = 'b', linestyle = '-', label = 'Скорость')
ln3, = ax[2].plot([], [], color = 'k', linestyle = '-', label = 'Энергия')
ln4, = ax[3].plot([], [], color = 'g', linestyle = '-', label = 'Давление')


def init():
    for i in range(4):
        ax[i].set_xlim(xdata[0], xdata[-1])
        ax[i].grid(True)
        ax[i].legend()
    
    ax[0].set_ylim(0, 5)
    ax[1].set_ylim(0, 1.2)
    ax[2].set_ylim(0, 5)
    ax[3].set_ylim(0, 5)
    return ln1,


def update(t):
    global ro, u, e
    for _ in range(frame_const):
        ro, u, e = nextLayer(ro, u, e)
        p = list(map(P, ro, u, e))
    ln1.set_data(xdata, ro)
    ln2.set_data(xdata, u)
    ln3.set_data(xdata, e)
    ln4.set_data(xdata, p)
    return ln1,


frame_const = 7
anim = FuncAnimation(fig, update, frames=tqdm_notebook(range(T//frame_const)), init_func=init, interval=30, blit=True)
html5video = anim.to_html5_video()
plt.close()
HTML(html5video)